In [10]:
import os
import numpy as np
import tensorflow as tf
import scipy.signal as sps
from scipy.io import wavfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score

def downsample_waveform(waveform, num_bins):
    waveform = np.array(waveform)
    original_length = len(waveform)
    points_per_bin = original_length // num_bins
    downsampled_waveform = np.zeros(num_bins)
    for i in range(num_bins):
        start_index = i * points_per_bin
        end_index = start_index + points_per_bin
        downsampled_waveform[i] = waveform[start_index:end_index].mean()
    return downsampled_waveform.tolist()

def add_white_noise(audio):
    #generate noise and the scalar multiplier
    noise = tf.random.uniform(shape=tf.shape(audio), minval=-1, maxval=1)
    noise_scalar = tf.random.uniform(shape=[1], minval=0, maxval=0.2)
    # add them to the original audio
    audio_with_noise = audio + (noise * noise_scalar)
    # final clip the values to ensure they are still between -1 and 1
    audio_with_noise = tf.clip_by_value(audio_with_noise, clip_value_min=-1, clip_value_max=1)
    return audio_with_noise

def extract_features(audio_file_path, window_size=1024, overlap=0, num_bins=16):
    sample_rate, audio_data = wavfile.read(audio_file_path)
    resampled_audio = sps.resample(audio_data, sample_rate)
    # Add white noise to the audio
    augmented_audio = add_white_noise(resampled_audio)
    step_size = window_size - overlap
    num_windows = (len(augmented_audio) - window_size) // step_size + 1
    fft_results = []
    for i in range(num_windows):
        start_index = i * step_size
        end_index = start_index + window_size
        windowed_signal = augmented_audio[start_index:end_index]
        
        fft_result = np.fft.fft(windowed_signal)
        fft_result = fft_result[0:int(fft_result.shape[0] / 2)]
        fft_magnitude = np.abs(fft_result)
        fft_magnitude[0] = 0
        fft_magnitude = downsample_waveform(fft_magnitude, num_bins)
        fft_results.extend(fft_magnitude)
    return np.array(fft_results)

In [13]:
data_dir = os.path.join(os.getcwd(), "rgb_wavs", "rgb")

def load_dataset(data_dir):
    waveforms = []
    labels = []
    for dirname in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, dirname)
        if not dirname in labels:
            labels.append(dirname)
        wav_files = [os.path.join(label_dir, fname) for fname in os.listdir(label_dir)]
        feature_arr = []
        for wav_file in wav_files:
            xfeatures = extract_features(wav_file)
            feature_arr.append(xfeatures)
        waveforms.append(np.array(feature_arr))
        del feature_arr
    return np.array(waveforms), np.array(labels)

def load_data(data_dir, color="red"):
    labels = []
    feature_arr = []
    red = 0
    green = 0
    blue = 0
    red_dir = os.path.join(data_dir, "red")
    green_dir = os.path.join(data_dir, "green")
    blue_dir = os.path.join(data_dir, "blue")
    if color == "red":
        red = 1
    elif color == "green":
        green = 1
    elif color == "blue":
        blue = 1
    red_files = [os.path.join(red_dir, fname) for fname in os.listdir(red_dir)[:10]]
    for wav_file in red_files:
        xfeatures = extract_features(wav_file)
        feature_arr.append(xfeatures)
        labels.append(red)
    green_files = [os.path.join(green_dir, fname) for fname in os.listdir(green_dir)[:10]]
    for wav_file in green_files:
        xfeatures = extract_features(wav_file)
        feature_arr.append(xfeatures)
        labels.append(green)
    blue_files = [os.path.join(blue_dir, fname) for fname in os.listdir(blue_dir)[:10]]
    for wav_file in blue_files:
        xfeatures = extract_features(wav_file)
        feature_arr.append(xfeatures)
        labels.append(blue)
    return np.array(feature_arr), np.array(labels)


red_audio_data, red_labels = load_data(data_dir, color="red")
green_audio_data, green_labels = load_data(data_dir, color="green")
blue_audio_data, blue_labels = load_data(data_dir, color="blue")

print(np.shape(red_audio_data))
print(np.shape(red_labels))

(30, 112)
(30,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(red_audio_data, red_labels, test_size=0.2, random_state=42)

def normalize(array):
    min_val = array.min()
    max_val = array.max()
    normalized_array = (array - min_val) / (max_val - min_val)
    return normalized_array

X_norm_train = np.array([normalize(x) for x in X_train])
print(np.shape(X_norm_train))
print(np.shape(y_train))

(24, 112)
(24,)


In [29]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(112,), name="input_embedding"))
model.add(tf.keras.layers.Dense(12, activation="relu"))
model.add(tf.keras.layers.Dense(8, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_norm_train, y_train, epochs=40, batch_size=32, validation_split=0.2)


Epoch 1/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step - accuracy: 0.6842 - loss: 0.6872 - val_accuracy: 0.6000 - val_loss: 0.6839
Epoch 2/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6842 - loss: 0.6600 - val_accuracy: 0.6000 - val_loss: 0.6831
Epoch 3/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6842 - loss: 0.6496 - val_accuracy: 0.6000 - val_loss: 0.6832
Epoch 4/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6842 - loss: 0.6354 - val_accuracy: 0.6000 - val_loss: 0.6900
Epoch 5/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6842 - loss: 0.6232 - val_accuracy: 0.6000 - val_loss: 0.7062
Epoch 6/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6842 - loss: 0.6165 - val_accuracy: 0.6000 - val_loss: 0.7241
Epoch 7/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6842 - loss: 0.6138 - val_accuracy: 0.6000 - val_loss: 0.7334
Epoch 8/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6842 - loss: 0.6128 - val_accuracy: 0.6000 - val_loss: 0.7368

In [30]:
weights_and_biases = {}
for i, layer in enumerate(model.layers):
    weights, biases = layer.get_weights()
    weights_and_biases[f'w{i}'] = weights
    weights_and_biases[f'b{i}'] = biases

# Save the weights and biases to a file
np.savez('model_weights_biases.npz', **weights_and_biases)